In [1]:
import os
import time
import sys
import fileinput
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from itertools import product, combinations
from collections import OrderedDict
import pandas as pd

In [2]:
class Simulation(object):
	def __init__(self):
		self.events = []

In [3]:
class Event(object):
	def __init__(self):
		self.id_trigger = None
		self.id_simulatedEvent = None
		self.time = None
		self.initialEnergy = None
		self.depositedEnergy = None
		self.escapedEnergy = None
		self.depositedEnergy_NonSensitiveMaterial = None

		self.interactions = None
		self.hits = None
		self.particleInformation = {}

In [4]:
class Interactions(object):

	def __init__(self):

		self.interactionType = []
		self.ID_interaction = []
		self.ID_parentInteraction = []
		self.ID_detector = []
		self.timeStart = []
		self.x = []
		self.y = []
		self.z = []
		self.ID_parentParticleType = []
		self.x_newDirection_OriginalParticle = []
		self.y_newDirection_OriginalParticle = []
		self.z_newDirection_OriginalParticle = []
		self.x_polarization_OriginalParticle = []
		self.y_polarization_OriginalParticle = []
		self.z_polarization_OriginalParticle = []
		self.newKineticEnergy_OriginalParticle = []
		self.ID_childParticleType = []
		self.x_direction_NewParticle = []
		self.y_direction_NewParticle = []
		self.z_direction_NewParticle = []
		self.x_polarization_NewParticle = []
		self.y_polarization_NewParticle = []
		self.z_polarization_NewParticle = []
		self.newKineticEnergy_NewParticle = []

In [5]:
class Hits(object):

	def __init__(self):

		self.x = []
		self.y = []
		self.z = []
		self.energy = []
		self.detector = []

	def __str__(self):

		str = ""
		hits = zip(self.detector,self.x,self.y,self.z,self.energy)
		for hit in hits:
			str += "BLAK {:d}; {:f}; {:f}; {:f}; {:f}\n".format(*hit)
		return str

In [6]:
def parse(filename, skipPhoto=True):
	"""
	A function to parse the cosima output simulation file.  The function returns a simulation object.
	Example Usage:
	simulation = EventViewer.parse(filename)
	 """

	# Start an event counter
	currentEventNumber = 0

	# Loop through each line of the file
	for line in fileinput.input([filename]):

		# Create the first event
		if 'SE' in line and currentEventNumber == 0:

			# Create a new simulation object to store all of the events in this run
			simulation = Simulation()

			# Create a new event
			event = Event()

			# Create a new object to store the interactions for this event
			interactions = Interactions()

			# Create a new object to store the hits for this event
			hits = Hits()

			# Increment the event number
			currentEventNumber = currentEventNumber + 1

		# Store the existing event and create a new event
		elif 'SE' in line or 'EN' in line:

			# Store the interaction and hit objects in their respective event
			event.interactions = interactions
			event.hits = hits

			# Store the current event in the simulation object
			simulation.events.append(event)

			# Create a new event
			event = Event()

			# Create a new object to store the interactions for the new event
			interactions = Interactions()

			# Create a new object to store the hits for the new event
			hits = Hits()

			# Increment the event number
			currentEventNumber = currentEventNumber + 1

		# Get the event ID
		if 'ID' in line and currentEventNumber != 0:

			event.id_trigger = line.split()[1]
			event.id_simulatedEvent = line.split()[2]

		# Get the event time
		if 'TI' in line and currentEventNumber != 0:

			event.time = line.split()[1]

		# Get the total deposited energy
		if 'ED' in line and currentEventNumber != 0:

			event.depositedEnergy = line.split()[1]

		# Get the total escaped energy
		if 'EC' in line and currentEventNumber != 0:

			event.escapedEnergy = line.split()[1]

		# Get the total deposited energy in non-sensative material
		if 'NS' in line and currentEventNumber != 0:

			event.depositedEnergy_NonSensitiveMaterial = line.split()[1]

		# if 'IA' in line and 'PHOT' not in line:
		if 'IA' in line:

			# Skip photoelectric interactions
			if skipPhoto == True:
				if 'PHOT' in line:
					continue

			# Split the line
			LineContents = line.split(';')

			# Parse each line and place the extracted information into their respective arrays
			interactions.interactionType.append(LineContents[0].split()[1].split()[0])
			interactions.ID_interaction.append(LineContents[0].split()[2].split()[0])
			interactions.ID_parentInteraction.append(LineContents[1].split()[0])
			interactions.ID_detector.append(LineContents[2])
			interactions.timeStart.append(float(LineContents[3]))
			interactions.x.append(float(LineContents[4]))
			interactions.y.append(float(LineContents[5]))
			interactions.z.append(float(LineContents[6]))
			interactions.ID_parentParticleType.append(LineContents[7].split()[0])
			interactions.x_newDirection_OriginalParticle.append(float(LineContents[8]))
			interactions.y_newDirection_OriginalParticle.append(float(LineContents[9]))
			interactions.z_newDirection_OriginalParticle.append(float(LineContents[10]))
			interactions.x_polarization_OriginalParticle.append(LineContents[11])
			interactions.y_polarization_OriginalParticle.append(LineContents[12])
			interactions.z_polarization_OriginalParticle.append(LineContents[13])
			interactions.newKineticEnergy_OriginalParticle.append(LineContents[14])
			interactions.ID_childParticleType.append(LineContents[15])
			interactions.x_direction_NewParticle.append(float(LineContents[16]))
			interactions.y_direction_NewParticle.append(float(LineContents[17]))
			interactions.z_direction_NewParticle.append(float(LineContents[18]))
			interactions.x_polarization_NewParticle.append(LineContents[19])
			interactions.y_polarization_NewParticle.append(LineContents[20])
			interactions.z_polarization_NewParticle.append(LineContents[21])
			interactions.newKineticEnergy_NewParticle.append(LineContents[22].rstrip())

			if 'INIT' in line:
				event.initialEnergy = interactions.newKineticEnergy_NewParticle[-1]

			# Create a unique particle id to track parent and child particles
			ID_parentParticle = interactions.ID_parentInteraction[-1] + '_' + interactions.ID_parentParticleType[-1]
			ID_childParticle = interactions.ID_interaction[-1] + '_' + interactions.ID_childParticleType[-1]

			# if ID_childParticleType == '1':
			# 	ID_childParticle = ID_parentInteraction + '_' + ID_childParticleType
			# else:
			# 	ID_childParticle = ID_interaction + '_' + ID_childParticleType

			# Store the information for the individual particles associated with this interaction

			# Record the particle trajectory
			if ID_parentParticle in event.particleInformation:
				event.particleInformation[ID_parentParticle]['x'].append(interactions.x[-1])
				event.particleInformation[ID_parentParticle]['y'].append(interactions.y[-1])
				event.particleInformation[ID_parentParticle]['z'].append(interactions.z[-1])
				event.particleInformation[ID_parentParticle]['time'].append(interactions.timeStart[-1])
			else:
				event.particleInformation[ID_parentParticle] = {}
				event.particleInformation[ID_parentParticle]['x'] = [interactions.x[-1]]
				event.particleInformation[ID_parentParticle]['y'] = [interactions.y[-1]]
				event.particleInformation[ID_parentParticle]['z'] = [interactions.z[-1]]
				event.particleInformation[ID_parentParticle]['time'] = [interactions.timeStart[-1]]

			if ID_childParticle in event.particleInformation:
				event.particleInformation[ID_childParticle]['x'].append(interactions.x[-1])
				event.particleInformation[ID_childParticle]['y'].append(interactions.y[-1])
				event.particleInformation[ID_childParticle]['z'].append(interactions.z[-1])
				event.particleInformation[ID_childParticle]['time'].append(timeStart[-1])
			else:
				event.particleInformation[ID_childParticle] = {}
				event.particleInformation[ID_childParticle]['x'] = [interactions.x[-1]]
				event.particleInformation[ID_childParticle]['y'] = [interactions.y[-1]]
				event.particleInformation[ID_childParticle]['z'] = [interactions.z[-1]]
				event.particleInformation[ID_childParticle]['time'] = [interactions.timeStart[-1]]

		# Record the hit information
		if 'BLAK' in line:

			# Split the line
			LineContents = line.split(';')

			# Extract the hit information
			#hits.detector.append(str(LineContents[0].split(' ')[1]))
			hits.x.append(float(LineContents[1]))
			hits.y.append(float(LineContents[2]))
			hits.z.append(float(LineContents[3]))
			hits.energy.append(float(LineContents[4]))


	# Close the input file
	fileinput.close()

	return simulation

In [7]:
parse('FullConcreteModel.inc3.id1.sim')